# Neural Network:
A Neural Network is a computational model inspired by the structure and functioning of the human brain. 
- It is a powerful and versatile machine learning algorithm used for various tasks such as pattern recognition, classification, regression, and more. 
- The network consists of interconnected nodes called neurons, organized into layers - an input layer, one or more hidden layers, and an output layer. 
- Each neuron receives input data, applies a weighted sum, adds a bias term, and then passes the result through an activation function to produce an output.

Neural networks have shown remarkable success in tasks like image recognition, natural language processing, and other areas requiring sophisticated pattern recognition and decision-making capabilities.  

Neural networks, particularly deep learning models, have demonstrated remarkable performance in various medical image analysis and classification tasks, including breast cancer classification.

In [1]:
import seaborn as sns
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import VarianceThreshold
from sklearn.metrics import f1_score, precision_score, accuracy_score, recall_score, balanced_accuracy_score
import tensorflow as tf
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import Model
import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('breast-cancer.csv')
df.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [3]:
df.drop('id', axis=1, inplace=True)
df['diagnosis'] = (df['diagnosis'] == 'M').astype(int)
# Encode M (Malignant) as 1 & B (Benign) as 0.
df.head()

,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,1,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,1,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,1,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,1,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


## Training the Model:
### Case I: With All Features:

In [4]:
X = df.drop('diagnosis', axis = 1)
y = df['diagnosis']

initializer = tf.keras.initializers.RandomNormal(seed = 0)
def scheduler(epoch, lr):
    if epoch < 50:
        return lr
    else:
        return lr * tf.math.exp(-0.1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)
scaler = StandardScaler() # Create an instance of standard scaler
scaler.fit(X_train) # Fit it to the training data
X_train = scaler.transform(X_train) # Transform training data
X_test = scaler.transform(X_test) # Transform validation data
callback = tf.keras.callbacks.LearningRateScheduler(scheduler) # set the callback to our scheduler function
inputs = Input(shape = X.shape[-1])
x = Dense(16,activation='relu',kernel_initializer=tf.keras.initializers.RandomNormal())(inputs)
x = Dropout(0.4)(x)
x = Dense(16,activation='relu',kernel_initializer=tf.keras.initializers.RandomNormal())(inputs)
x = Dropout(0.4)(x)
x = Dense(25,activation='relu',kernel_initializer=tf.keras.initializers.RandomNormal())(x)
x = Dense(1, activation='sigmoid',kernel_initializer=tf.keras.initializers.RandomNormal())(x)
model = Model(inputs=inputs, outputs=x)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train,epochs=140)
predictions = model.predict(X_test)

Epoch 1/140
15/15 [==============================] - 1s 2ms/step - loss: 0.6892 - accuracy: 0.6681
Epoch 2/140
15/15 [==============================] - 0s 2ms/step - loss: 0.6716 - accuracy: 0.9407
Epoch 3/140
15/15 [==============================] - 0s 2ms/step - loss: 0.6346 - accuracy: 0.9209
Epoch 4/140
15/15 [==============================] - 0s 2ms/step - loss: 0.5687 - accuracy: 0.9319
Epoch 5/140
15/15 [==============================] - 0s 2ms/step - loss: 0.4901 - accuracy: 0.9407
Epoch 6/140
15/15 [==============================] - 0s 2ms/step - loss: 0.4106 - accuracy: 0.9516
Epoch 7/140
15/15 [==============================] - 0s 2ms/step - loss: 0.3260 - accuracy: 0.9604
Epoch 8/140
15/15 [==============================] - 0s 2ms/step - loss: 0.2543 - accuracy: 0.9648
Epoch 9/140
15/15 [==============================] - 0s 2ms/step - loss: 0.1886 - accuracy: 0.9736
Epoch 10/140
15/15 [==============================] - 0s 2ms/step - loss: 0.1586 - accuracy: 0.9736
Epoch 11/

15/15 [==============================] - 0s 2ms/step - loss: 0.0417 - accuracy: 0.9890
Epoch 84/140
15/15 [==============================] - 0s 2ms/step - loss: 0.0368 - accuracy: 0.9934
Epoch 85/140
15/15 [==============================] - 0s 2ms/step - loss: 0.0333 - accuracy: 0.9912
Epoch 86/140
15/15 [==============================] - 0s 2ms/step - loss: 0.0292 - accuracy: 0.9956
Epoch 87/140
15/15 [==============================] - 0s 2ms/step - loss: 0.0341 - accuracy: 0.9912
Epoch 88/140
15/15 [==============================] - 0s 2ms/step - loss: 0.0285 - accuracy: 0.9956
Epoch 89/140
15/15 [==============================] - 0s 2ms/step - loss: 0.0265 - accuracy: 0.9956
Epoch 90/140
15/15 [==============================] - 0s 2ms/step - loss: 0.0356 - accuracy: 0.9912
Epoch 91/140
15/15 [==============================] - 0s 2ms/step - loss: 0.0365 - accuracy: 0.9890
Epoch 92/140
15/15 [==============================] - 0s 2ms/step - loss: 0.0290 - accuracy: 0.9912
Epoch 93/140


In [5]:
# Generate binary predictions for the test data using the trained neural network model
predictions = model.predict(X_test)
# convert predicted probabilities to binary predictions Note that a threshold of 0.4 was used as recall is more important for medical diagnosis
predictions = (predictions > 0.4) 

# Calculate evaluation metrics using the binary predictions and true labels for the test data
accuracy = accuracy_score(y_test, predictions)
f1 = f1_score(y_test, predictions)
precision = precision_score(y_test, predictions)
recall = recall_score(y_test, predictions)
balanced_accuracy = balanced_accuracy_score(y_test, predictions)

# Create a pandas dataframe containing the evaluation metric values as a single row
nn_df = pd.DataFrame([[accuracy, f1, precision, recall, balanced_accuracy]], 
                     columns=['Accuracy', 'F1-Score', 'Precision', 'Recall', 'Balanced Accuracy'])
nn_df.index = ['Neural Network (All Features)']
nn_df.sort_values(by='F1-Score',ascending=False).style.background_gradient(cmap = sns.color_palette("ch:s=-.2,r=.6", as_cmap=True))

4/4 [==============================] - 0s 2ms/step


,Accuracy,F1-Score,Precision,Recall,Balanced Accuracy
Neural Network (All Features),0.982456,0.976744,0.976744,0.976744,0.981330


### Case II: Manual Feature Selection:

In [6]:
# Get the absolute value of the correlation:
corr = df.corr()
cor_target = abs(corr["diagnosis"])
cor_target

diagnosis                  1.000000
radius_mean                0.730029
texture_mean               0.415185
perimeter_mean             0.742636
area_mean                  0.708984
smoothness_mean            0.358560
compactness_mean           0.596534
concavity_mean             0.696360
concave points_mean        0.776614
symmetry_mean              0.330499
fractal_dimension_mean     0.012838
radius_se                  0.567134
texture_se                 0.008303
perimeter_se               0.556141
area_se                    0.548236
smoothness_se              0.067016
compactness_se             0.292999
concavity_se               0.253730
concave points_se          0.408042
symmetry_se                0.006522
fractal_dimension_se       0.077972
radius_worst               0.776454
texture_worst              0.456903
perimeter_worst            0.782914
area_worst                 0.733825
smoothness_worst           0.421465
compactness_worst          0.590998
concavity_worst            0

In [7]:
# Select highly correlated features (threshold = 0.5):
relevant_features = cor_target[cor_target > 0.7]
# Collect the names of the features:
names = [index for index, value in relevant_features.iteritems()]
# Drop the target variable from the results:
names.remove('diagnosis')

In [8]:
X = df[names]
y = df['diagnosis']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)
scaler = StandardScaler() # Create an instance of standard scaler
scaler.fit(X_train) # Fit it to the training data
X_train = scaler.transform(X_train) # Transform training data
X_test = scaler.transform(X_test) # Transform validation data

In [9]:
callback = tf.keras.callbacks.LearningRateScheduler(scheduler) # set the callback to our scheduler function
inputs = Input(shape = X.shape[-1])
x = Dense(16,activation='relu',kernel_initializer=tf.keras.initializers.RandomNormal())(inputs)
x = Dropout(0.4)(x)
x = Dense(16,activation='relu',kernel_initializer=tf.keras.initializers.RandomNormal())(inputs)
x = Dropout(0.4)(x)
x = Dense(25,activation='relu',kernel_initializer=tf.keras.initializers.RandomNormal())(x)
x = Dense(1, activation='sigmoid',kernel_initializer=tf.keras.initializers.RandomNormal())(x)
model = Model(inputs=inputs, outputs=x)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train,epochs=140)
predictions = model.predict(X_test)

Epoch 1/140
15/15 [==============================] - 1s 2ms/step - loss: 0.6899 - accuracy: 0.7802
Epoch 2/140
15/15 [==============================] - 0s 2ms/step - loss: 0.6778 - accuracy: 0.8901
Epoch 3/140
15/15 [==============================] - 0s 2ms/step - loss: 0.6488 - accuracy: 0.9165
Epoch 4/140
15/15 [==============================] - 0s 2ms/step - loss: 0.5919 - accuracy: 0.9187
Epoch 5/140
15/15 [==============================] - 0s 3ms/step - loss: 0.5100 - accuracy: 0.9297
Epoch 6/140
15/15 [==============================] - 0s 2ms/step - loss: 0.4032 - accuracy: 0.9209
Epoch 7/140
15/15 [==============================] - 0s 2ms/step - loss: 0.3171 - accuracy: 0.9231
Epoch 8/140
15/15 [==============================] - 0s 3ms/step - loss: 0.2513 - accuracy: 0.9275
Epoch 9/140
15/15 [==============================] - 0s 3ms/step - loss: 0.2210 - accuracy: 0.9187
Epoch 10/140
15/15 [==============================] - 0s 2ms/step - loss: 0.2039 - accuracy: 0.9253
Epoch 11/

15/15 [==============================] - 0s 2ms/step - loss: 0.1203 - accuracy: 0.9560
Epoch 84/140
15/15 [==============================] - 0s 3ms/step - loss: 0.1142 - accuracy: 0.9648
Epoch 85/140
15/15 [==============================] - 0s 3ms/step - loss: 0.1159 - accuracy: 0.9626
Epoch 86/140
15/15 [==============================] - 0s 3ms/step - loss: 0.1108 - accuracy: 0.9560
Epoch 87/140
15/15 [==============================] - 0s 3ms/step - loss: 0.1113 - accuracy: 0.9626
Epoch 88/140
15/15 [==============================] - 0s 2ms/step - loss: 0.1178 - accuracy: 0.9626
Epoch 89/140
15/15 [==============================] - 0s 2ms/step - loss: 0.1160 - accuracy: 0.9582
Epoch 90/140
15/15 [==============================] - 0s 2ms/step - loss: 0.1156 - accuracy: 0.9626
Epoch 91/140
15/15 [==============================] - 0s 2ms/step - loss: 0.1121 - accuracy: 0.9626
Epoch 92/140
15/15 [==============================] - 0s 2ms/step - loss: 0.1241 - accuracy: 0.9582
Epoch 93/140


In [10]:
# Generate binary predictions for the test data using the trained neural network model
predictions = model.predict(X_test)
# convert predicted probabilities to binary predictions Note that a threshold of 0.4 was used as recall is more important for medical diagnosis
predictions = (predictions > 0.4) 

# Calculate evaluation metrics using the binary predictions and true labels for the test data
accuracy = accuracy_score(y_test, predictions)
f1 = f1_score(y_test, predictions)
precision = precision_score(y_test, predictions)
recall = recall_score(y_test, predictions)
balanced_accuracy = balanced_accuracy_score(y_test, predictions)

# Create a pandas dataframe containing the evaluation metric values as a single row
nn_df1 = pd.DataFrame([[accuracy, f1, precision, recall, balanced_accuracy]], 
                     columns=['Accuracy', 'F1-Score', 'Precision', 'Recall', 'Balanced Accuracy'])
nn_df1.index = ['Neural Network (Manual)']
nn_df = nn_df.append(nn_df1)
nn_df.sort_values(by='F1-Score',ascending=False).style.background_gradient(cmap = sns.color_palette("ch:s=-.2,r=.6", as_cmap=True))

4/4 [==============================] - 0s 1ms/step


,Accuracy,F1-Score,Precision,Recall,Balanced Accuracy
Neural Network (All Features),0.982456,0.976744,0.976744,0.976744,0.981330
Neural Network (Manual),0.964912,0.953488,0.953488,0.953488,0.962660


### Case III: Variance Thresholding:
This method removes features with low variance, assuming that features with little variation provide limited information for the model.

In [11]:
X = df.drop('diagnosis', axis = 1)
y = df['diagnosis']
threshold_value = 0.03
variance_threshold = VarianceThreshold(threshold=threshold_value)
# Fit the VarianceThreshold object to the data.
variance_threshold.fit(X)
# Transform the data to remove low-variance features.
X_high_variance = variance_threshold.transform(X)
# The "get_support" method returns a boolean mask of the selected features.
selected_features_mask = variance_threshold.get_support()

# You can use this mask to get the indices or names of the selected features.
selected_feature_indices = np.where(selected_features_mask)[0]
selected_feature_names = X.columns[selected_feature_indices]

In [12]:
X = df[selected_feature_names]
y = df['diagnosis']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)
scaler = StandardScaler() # Create an instance of standard scaler
scaler.fit(X_train) # Fit it to the training data
X_train = scaler.transform(X_train) # Transform training data
X_test = scaler.transform(X_test) # Transform validation data

In [13]:
callback = tf.keras.callbacks.LearningRateScheduler(scheduler) # set the callback to our scheduler function
inputs = Input(shape = X.shape[-1])
x = Dense(16,activation='relu',kernel_initializer=tf.keras.initializers.RandomNormal())(inputs)
x = Dropout(0.4)(x)
x = Dense(16,activation='relu',kernel_initializer=tf.keras.initializers.RandomNormal())(inputs)
x = Dropout(0.4)(x)
x = Dense(25,activation='relu',kernel_initializer=tf.keras.initializers.RandomNormal())(x)
x = Dense(1, activation='sigmoid',kernel_initializer=tf.keras.initializers.RandomNormal())(x)
model = Model(inputs=inputs, outputs=x)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train,epochs=140)
predictions = model.predict(X_test)

Epoch 1/140
15/15 [==============================] - 1s 2ms/step - loss: 0.6891 - accuracy: 0.7319
Epoch 2/140
15/15 [==============================] - 0s 2ms/step - loss: 0.6742 - accuracy: 0.8198
Epoch 3/140
15/15 [==============================] - 0s 2ms/step - loss: 0.6384 - accuracy: 0.8681
Epoch 4/140
15/15 [==============================] - 0s 2ms/step - loss: 0.5680 - accuracy: 0.9033
Epoch 5/140
15/15 [==============================] - 0s 2ms/step - loss: 0.4549 - accuracy: 0.9297
Epoch 6/140
15/15 [==============================] - 0s 3ms/step - loss: 0.3404 - accuracy: 0.9429
Epoch 7/140
15/15 [==============================] - 0s 2ms/step - loss: 0.2600 - accuracy: 0.9363
Epoch 8/140
15/15 [==============================] - 0s 2ms/step - loss: 0.2161 - accuracy: 0.9385
Epoch 9/140
15/15 [==============================] - 0s 2ms/step - loss: 0.1819 - accuracy: 0.9407
Epoch 10/140
15/15 [==============================] - 0s 2ms/step - loss: 0.1696 - accuracy: 0.9429
Epoch 11/

15/15 [==============================] - 0s 2ms/step - loss: 0.0853 - accuracy: 0.9648
Epoch 84/140
15/15 [==============================] - 0s 2ms/step - loss: 0.0894 - accuracy: 0.9714
Epoch 85/140
15/15 [==============================] - 0s 2ms/step - loss: 0.0948 - accuracy: 0.9670
Epoch 86/140
15/15 [==============================] - 0s 2ms/step - loss: 0.0882 - accuracy: 0.9626
Epoch 87/140
15/15 [==============================] - 0s 2ms/step - loss: 0.0929 - accuracy: 0.9626
Epoch 88/140
15/15 [==============================] - 0s 2ms/step - loss: 0.0841 - accuracy: 0.9714
Epoch 89/140
15/15 [==============================] - 0s 2ms/step - loss: 0.1019 - accuracy: 0.9692
Epoch 90/140
15/15 [==============================] - 0s 2ms/step - loss: 0.0871 - accuracy: 0.9648
Epoch 91/140
15/15 [==============================] - 0s 2ms/step - loss: 0.0842 - accuracy: 0.9626
Epoch 92/140
15/15 [==============================] - 0s 2ms/step - loss: 0.0955 - accuracy: 0.9648
Epoch 93/140


In [14]:
# Generate binary predictions for the test data using the trained neural network model
predictions = model.predict(X_test)
# convert predicted probabilities to binary predictions Note that a threshold of 0.4 was used as recall is more important for medical diagnosis
predictions = (predictions > 0.4) 

# Calculate evaluation metrics using the binary predictions and true labels for the test data
accuracy = accuracy_score(y_test, predictions)
f1 = f1_score(y_test, predictions)
precision = precision_score(y_test, predictions)
recall = recall_score(y_test, predictions)
balanced_accuracy = balanced_accuracy_score(y_test, predictions)

# Create a pandas dataframe containing the evaluation metric values as a single row
nn_df2 = pd.DataFrame([[accuracy, f1, precision, recall, balanced_accuracy]], 
                     columns=['Accuracy', 'F1-Score', 'Precision', 'Recall', 'Balanced Accuracy'])
nn_df2.index = ['Neural Network (VT)']
nn_df = nn_df.append(nn_df2)
nn_df.sort_values(by='F1-Score',ascending=False).style.background_gradient(cmap = sns.color_palette("ch:s=-.2,r=.6", as_cmap=True))

4/4 [==============================] - 0s 2ms/step


,Accuracy,F1-Score,Precision,Recall,Balanced Accuracy
Neural Network (All Features),0.982456,0.976744,0.976744,0.976744,0.981330
Neural Network (Manual),0.964912,0.953488,0.953488,0.953488,0.962660
Neural Network (VT),0.956140,0.941176,0.952381,0.930233,0.951032
